# Quantifying Pharmacy Deserts with Xarray-spatial

In this notebook, we'll demonstrate using Xarray-spatial to find and map the locations of pharmacy deserts in the USA.

### Setup:

First, we'll import some basic data and geospatial data-related packages. We'll also bring in some datashader functions for aggregation and rendering. And, finally, we'll import the Xarray-spatial functions we'll be using. 

In [3]:
#!pip install geopandas

You should consider upgrading via the '/Users/marigaldina/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [4]:
!pip install --upgrade pip

     |████████████████████████████████| 1.5 MB 4.2 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 21.0.1
    Uninstalling pip-21.0.1:
      Successfully uninstalled pip-21.0.1


In [5]:
!pip install spatialpandas

     |████████████████████████████████| 77 kB 1.7 MB/s eta 0:00:011
     |████████████████████████████████| 45 kB 4.0 MB/s  eta 0:00:01
     |████████████████████████████████| 15.7 MB 10.3 MB/s eta 0:00:01


In [7]:
!pip install datashader

     |████████████████████████████████| 15.8 MB 4.3 MB/s eta 0:00:01    |█████████▉                      | 4.9 MB 4.3 MB/s eta 0:00:03
     |████████████████████████████████| 76 kB 2.7 MB/s eta 0:00:01
  Created wheel for datashape: filename=datashape-0.5.2-py3-none-any.whl size=59428 sha256=06a59ef9dac3c0df8d57be8e4489fab25ef9ec7bc11150ff60c6010e328ae7a5
  Stored in directory: /Users/marigaldina/Library/Caches/pip/wheels/b5/b7/80/333a5c3312ed4cd54f5d5b869868c14e0c6002cb5c7238b52d
Successfully built datashape


In [8]:
!pip install xarray-spatial

     |████████████████████████████████| 2.0 MB 4.3 MB/s eta 0:00:01
     |████████████████████████████████| 705 kB 4.9 MB/s eta 0:00:01
     |████████████████████████████████| 964 kB 7.0 MB/s eta 0:00:01
     |████████████████████████████████| 111 kB 4.2 MB/s eta 0:00:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 0.5.2
    Uninstalling fsspec-0.5.2:
      Successfully uninstalled fsspec-0.5.2
  Attempting uninstall: cloudpickle
    Found existing installation: cloudpickle 1.2.2
    Uninstalling cloudpickle-1.2.2:
      Successfully uninstalled cloudpickle-1.2.2
  Attempting uninstall: tblib
    Found existing installation: tblib 1.4.0
    Uninstalling tblib-1.4.0:
      Successfully uninstalled tblib-1.4.0
  Attempting uninstall: dask
    Found existing installation: dask 2.5.2
    Uninstalling dask-2.5.2:
      Successfully uninstalled dask-2.5.2


  Attempting uninstall: pyct
    Found existing installation: pyct 0.4.8
    Uninstalling pyct-0.4.8:
      Successfully uninstalled pyct-0.4.8
  Attempting uninstall: distributed
    Found existing installation: distributed 2.5.2
    Uninstalling distributed-2.5.2:
      Successfully uninstalled distributed-2.5.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 3.3.6 requires pyqt5<5.13; python_version >= "3", which is not installed.
spyder 3.3.6 requires pyqtwebengine<5.13; python_version >= "3", which is not installed.


In [9]:
import numpy as np
import pandas as pd
import geopandas as gpd
from spatialpandas import GeoDataFrame

import datashader as ds
from datashader.transfer_functions import stack
from datashader.transfer_functions import shade
from datashader.transfer_functions import set_background
from datashader.colors import inferno

from xrspatial.classify import natural_breaks
from xrspatial.classify import binary
from xrspatial import proximity

## Load data

Let's load up the data we'll be playing with today: US pharmacy facilities, US block group data, and a US counties shapefile.

### Data Sources
- [Facilities CSV](https://rxopen.org/)
- [US Block Groups Shapefile](https://hub.arcgis.com/datasets/esri::usa-block-groups/data?geometry=-166.940%2C28.846%2C167.571%2C67.170)
- [US Counties Shapefile](https://hub.arcgis.com/datasets/48f9af87daa241c4b267c5931ad3b226_0?geometry=-166.940%2C28.846%2C167.571%2C67.170)

First, we'll create a DataFrame of pharmacy facility locations and clean that up a bit and set the coordinates.

In [13]:
# Load Pharmacies and add out x, y fields based on CalcLocation
pharmacy_df = pd.read_csv("./data/facility.csv")
coords = pharmacy_df['CalcLocation'].str.split(',', expand=True)
pharmacy_df['y'] = np.array(coords[0], dtype='float64')
pharmacy_df['x'] = np.array(coords[1], dtype='float64')
pharmacy_df

,CalcLocation,Name,Address,City,State,Zip,formated_phone,Status,Icon,Type,y,x
0,"32.32474700,-90.92676500",FMCNA - DELTA,104 RAILROAD AVE,DELTA,LA,71233,Please call patient admission line,open,small_green,Dialysis Center,32.324747,-90.926765
1,"46.00736300,-91.49085700",FMC - HAYWARD,10342 DYNO DR,HAYWARD,WI,54843,Please call patient admission line,open,small_green,Dialysis Center,46.007363,-91.490857
2,"34.79047900,-96.68271700",FMCNA - ADA DIALYSIS CTR,324 NW J.A. RICHARDSON LOOP,ADA,OK,74820,Please call patient admission line,open,small_green,Dialysis Center,34.790479,-96.682717
3,"38.92114100,-104.78556200",062563 LIBERTY DIALYSIS - COLORADO SPRINGS NOR...,2180 HOLLOWBROOK DR,COLORADO SPRINGS,CO,80918,Please call patient admission line,open,small_green,Dialysis Center,38.921141,-104.785562
4,"29.98721000,-95.49631600",CHAMPIONS DIALYSIS,4427 Cypress Creek Pkwy,HOUSTON,TX,77068,Please call patient admission line,open,small_green,Dialysis Center,29.987210,-95.496316
...,...,...,...,...,...,...,...,...,...,...,...,...
70713,"39.96783900,-75.20336900",Nextdoor Community Pharmacy,814 1/2 N 40th St,Philadelphia,PA,191044802,215-921-3191,open,small_green,Pharmacy,39.967839,-75.203369
70714,"40.52023400,-75.56858200",Macungie Community Pharmacy Inc,6465 Village Ln Ste 9,Macungie,PA,180628474,978-863-8342,open,small_green,Pharmacy,40.520234,-75.568582
70715,"40.45670300,-79.92978000",Capsule Pharmacy,5846 Ellsworth Ave,Pittsburgh,PA,152321708,412-388-4918,open,small_green,Pharmacy,40.456703,-79.929780
70716,"39.83663100,-75.44242700",Wellness Family Pharmacy,3250 Chichester Ave,Upper Chichester,PA,190613250,610-234-2922,open,small_green,Pharmacy,39.836631,-75.442427


Next, we'll do the same for the US block group data, but we'll use geopandas for this, because it involves spatial data.
We'll also group together all age groups above 65 to simplify things a bit. 

In [14]:
# Load Census Block Groups and Calculate Percent over 65 years-old
blockgroup_df = gpd.read_file("zip://./data/USA_Block_Groups-shp.zip")
blockgroup_df = GeoDataFrame(blockgroup_df, geometry='geometry')
blockgroup_df['ABOVE_65'] = blockgroup_df[['AGE_65_74', 'AGE_75_84', 'AGE_85_UP']].sum(axis=1) 
blockgroup_df['PCT_ABOVE_65'] = blockgroup_df['ABOVE_65'] / blockgroup_df['POP2010']
blockgroup_df

DriverError: '/vsizip/./data/USA_Block_Groups-shp.zip' does not exist in the file system, and is not recognized as a supported dataset name.

And, we'll do the same again for the counties data.

In [ ]:
# Load Census County and Calculate Percent over 65 years-old
county_df = gpd.read_file("zip://./data/USA_Counties-shp.zip")
county_df = GeoDataFrame(county_df, geometry='geometry')
county_df['ZONE_ID'] = county_df['OBJECTID'].astype(np.int16)
county_df

## Define Study Area and aggregate:

Now we're ready to set up our aggregate rasters (data that's been aggregated into the regular row-column cell format of a raster image) and then perform calculations on those rasters and combine them in informative ways.

The first raster we'll set up is the counties mask; we'll use this one to make our other rasters fit with county shapes.

- We set a range for for our x and y coordinates in longitude and latitude values.
- We use datashder's Canvas object as an easy-to-use frame to build up a raster with and aggregate data into.
- Finally, we aggregate the county shapes - polygons - into that raster. 

A quick shade and transformation with datashader functions visualizes our mask set-up.

In [ ]:
x_range = (-124.848974, -66.885444)
y_range = (24.396308, 49.384358)

W = 1600
H = 800

cvs = ds.Canvas(plot_width=W, plot_height=H,
                x_range=x_range, y_range=y_range)

county_mask = cvs.polygons(county_df, geometry='geometry')
set_background(shade(county_mask, cmap='#333333', alpha=255), 'black')

Now we can set up the other rasters and use some Xarray-spatial functions to quantify pharmacy distances, age groups, and county data, and how they relate to each other.

### Create a "Distance to Nearest Pharmacy" Raster Layer & Classify into 4 Groups

First, we'll create a pharmacy locations raster layer by aggregating our pharmacies DataFrame from above.
Using `proximity`, we can create transform this into a raster of the distances for each point to its nearest pharmacy.
We can also further orgnaize this by applying Xarray-spatial's `natural_breaks` reclassification function. This will break up our proximity data into neat bins so it's easier to visualize and interpret.

In [ ]:
pharmacy_raster = cvs.points(pharmacy_df, 'x', 'y')
proximity_raster = proximity(pharmacy_raster, distance_metric='GREAT_CIRCLE').where(county_mask)
proximity_raster.data[~np.isfinite(proximity_raster.data)] = 0.0

proximity_classifed = natural_breaks(proximity_raster, 20000, k=4).where(county_mask)

image_pharmacy = shade(proximity_classifed, cmap=inferno, alpha=255)
image_pharmacy = set_background(image_pharmacy, 'black')
image_pharmacy

### Create an Age Layer  & Classify into 4 Groups

Next, we'll create an age groups raster from the block group DataFrame above.
We'll also reclassify this with `natural_breaks` to make it easier to map.

In [ ]:
age_raster = cvs.polygons(blockgroup_df, geometry='geometry', agg=ds.mean('PCT_ABOVE_65'))
age_raster.data[~np.isfinite(age_raster.data)] = 0.0

age_classifed = natural_breaks(age_raster, 20000, k=4).where(county_mask)

age_image = shade(age_classifed, cmap=inferno, alpha=255)
age_image = set_background(age_image, 'black')
age_image

### Combine layers to highlight seniors at risk from pharmacy deserts

Finally, we'll combine the county, age, and pharmacy distance raster layers to highlight counties where there are more seniors who are at greater distance from pharmacies.

In [ ]:
pharmacy_deserts = binary(proximity_classifed, [3])
older_regions = binary(age_classifed, [3])
target_deserts = (pharmacy_deserts * older_regions).where(county_mask)
target_deserts_img = shade(target_deserts, cmap=['#333333', 'fuchsia'], alpha=255, how='linear')
set_background(target_deserts_img, 'black')

###  Summarize seniors at risk by county: the Zonal Statistics function

The renderings above are great, but we might also want to summarize the quantitative info we just generated as a table with summary statistics. Xarray-spatial's zonal statisitics function lets us do that. 

`Zonal statistics` operates on a values raster by applying a zones aggregate to it. This is defined by creating an integer aggregate corresponding to the shape of the values raster, but with the value set at each cell to a non-zero integer that is the id of the zone we wish to put that cell in. `Zonal_stats` gathers the values for all cells assigned to each zone_id and generates summary statistics for each one. The output is a Pandas dataframe with the calculation results in columns and with a row for each zone.

#### Setup:

First, we'll set up our counties raster as a background for the statistics we're about to generate and shade with some fun colors just to take a look at it now (just because we want a table of values, doesn't mean we can't also visualize it if we like).

In [ ]:
from datashader.colors import Set1

counties_raster = cvs.polygons(county_df, geometry='geometry', agg=ds.max('OBJECTID'))
counties_image = shade(counties_raster, cmap=Set1, alpha=225, how='linear')
set_background(counties_image, 'black')

Now, let's generate those statistics.

After a quick clean-up, we'll put our rasters in as arguments: we put counties as the zones, put target deserts as the values  
to segregate into those zones, and provide a dict with our desired statistics function to perform.

Note: `zonal_stats` comes with its own set of default calculations it will perform if the 3rd "zonal_funcs" argument is left empty, and please do feel free to clear that out and see what happens.
But, for our purposes, all we need is a simple mean calculation, so we'll set that into a dict, which we'll put into zonal_stats.



In [ ]:
from xrspatial import zonal_stats

# zones
counties_raster.data = counties_raster.data.astype(np.int64)

# values to summarize
target_deserts.data = target_deserts.data.astype(np.int8)

# summary functions
zonal_funcs = dict(pharmacy_desert_mean=lambda z: z.mean())

# execute summary functions on each zone and take top 10
results = zonal_stats(counties_raster, target_deserts, zonal_funcs)

Now we can merge this results DataFrame back into the original counties one. 

In [ ]:
cols = ['pharmacy_desert_mean', 'NAME', 'STATE_FIPS', 'geometry']
final_df = pd.merge(county_df, results, left_on='ZONE_ID', right_index=True)[cols]
final_df.nlargest(10, 'pharmacy_desert_mean')

Finally, we'll rasterize and shade these to visualize the results. We've highlighted the 100 counties with the highest pharmacy desert measures for seniors and shaded them by how much distance away, on average, the pharamcies are.

In [ ]:
from xrspatial import hillshade

counties_raster = cvs.polygons(county_df, geometry='geometry', agg=ds.max('OBJECTID'))

desert_raster = cvs.polygons(final_df.nlargest(75, 'pharmacy_desert_mean'),
                             geometry='geometry',
                             agg=ds.mean('pharmacy_desert_mean'))

county_mask = cvs.polygons(county_df, geometry='geometry')

In [ ]:
img = stack(
    shade(county_mask, cmap=['#0f9425'], alpha=255),
    shade(counties_raster, cmap=['green', '#ffffff'], alpha=25),
    shade(desert_raster, cmap=['white','#ff0000'], alpha=255),
    shade(hillshade(desert_raster), cmap=['green', '#ff0000'], alpha=150),
)
set_background(img, 'black')